In [1]:
!pip install --upgrade pinecone

In [2]:
from sentence_transformers import SentenceTransformer
import pinecone
import unicodedata

In [25]:
# Preprocess text
def preprocess_text(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')

# Generate embeddings
model = SentenceTransformer('BAAI/bge-large-en-v1.5')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

d:\Anaconda\envs\agents\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\VIKAS\.cache\huggingface\hub\models--BAAI--bge-large-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [16]:
import json

# Read the JSON file
with open('combined_records.json', 'r') as file:
    combined_records = json.load(file)

# Print the data to verify
print(combined_records)


[{'message_id': 'messg_0', 'conversation_id': 'conv_0', 'platform': 'LinkedIn', 'connection_name': 'Sri Teja Vure', 'message_sender': 'Vikas Reddy Venkannagari', 'message_date': '2025-04-22 13:24:00', 'message_text': 'Hey Sri Teja, I’m Vikas, a current grad student at UMBC, applying for Software Engineer role at Salesforce. I’d love your take—would you be open to a referral? I bring over 1.5 years of relevant experience and am happy to share my resume. Thanks!', 'connection_title': 'Software Engineering MTS @ Salesforce | CS Grad - Stony Brook University | Ex-TikTok, Oracle'}, {'message_id': 'messg_0', 'conversation_id': 'conv_1', 'platform': 'LinkedIn', 'connection_name': 'Sri Teja Vure', 'message_sender': 'Sri Teja Vure', 'message_date': '2025-04-23 16:38:00', 'message_text': 'Hi, sure, which job id are you looking for?', 'connection_title': 'Software Engineering MTS @ Salesforce | CS Grad - Stony Brook University | Ex-TikTok, Oracle'}, {'message_id': 'messg_0', 'conversation_id': 'c

In [29]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="pcsk_3ksvr_4eNPjVFcrNfS14446oiFLNsVgCWeHmXSHmGDXqg6VFsf9sucLKWsp1VEztkDQgQ")

In [26]:
    
embeddings = model.encode([preprocess_text(msg['message_text']) for msg in combined_records])

In [27]:
# Prepare vectors
vectors = [
    {
        "id": msg["message_id"],
        "values": embeddings[i].tolist(),
        "metadata": {
            "conversation_id": msg["conversation_id"],
            "platform": msg["platform"],
            "message_sender": msg["message_sender"],
            "message_date": msg["message_date"],
            "connection_name": msg["connection_name"],
            "connection_title": msg["connection_title"],
            "message_text": msg["message_text"]
        }
    }
    for i, msg in enumerate(combined_records)
]

In [ ]:
# if you want to create a new index, uncomment the following lines

# pc.create_index(
#     name="lnkdn2",
#     dimension=1024,
#     metric="cosine",
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     )
# )

{
    "name": "lnkdn",
    "metric": "cosine",
    "host": "lnkdn-xrghoiu.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 2048,
    "deletion_protection": "disabled",
    "tags": null
}

In [31]:
index = pc.Index("lkdn2")

In [32]:
index.upsert(vectors)

{'upserted_count': 430}

In [34]:
# Step 3: Generate Query Embedding
query = "can you refer to amazon"  # Your query text
query_embedding = model.encode([query], show_progress_bar=False)[0].tolist()  # Convert to list for Pinecone

# Step 4: Query Pinecone for Top-k Results
k = 5  # Number of top results to retrieve
query_response = index.query(
    vector=query_embedding,
    top_k=k,
    include_values=False,  # Set to True if you need the embedding vectors
    include_metadata=True   # Set to True if you stored metadata (e.g., original text)
)

# Step 5: Process and Display Results
print(f"Top {k} results for query: '{query}'")
for match in query_response["matches"]:
    print(f"ID: {match['id']}, Score: {match['score']:.4f}")
    if "metadata" in match and match["metadata"]:
        print(f"Metadata: {match['metadata']}")
    print("-" * 50)

Top 5 results for query: 'can you refer to amazon'
ID: messg_124, Score: 0.7204
Metadata: {'connection_name': 'Anusha Battu', 'connection_title': 'SDE @ Amazon AWS| Ex-SDE Intern @Apple | Texas A&M University College Station MSCS', 'conversation_id': 'conv_0', 'message_date': '2025-02-26 14:17:00', 'message_sender': 'Vikas Reddy Venkannagari', 'message_text': 'Hi Anusha, I’m interested in applying for an SDE role at Amazon [job id: 2850418] and would love a referral if possible. Let me know if you’d be open to it—happy to share my resume. Thanks!(Edited)', 'platform': 'LinkedIn'}
--------------------------------------------------
ID: messg_125, Score: 0.7199
Metadata: {'connection_name': 'Kapil Gupta', 'connection_title': 'Engineering Manager @Amazon | Ex-Nvidia', 'conversation_id': 'conv_0', 'message_date': '2025-02-26 14:15:00', 'message_sender': 'Vikas Reddy Venkannagari', 'message_text': 'Hi Kapil, I’m interested in applying for an SDE role at Amazon [job id: 2850418] and would lov